In [1]:
# Expenses And  Opportunity Cost are Corrected it seems
# Need way to reduce Expenses - No Control Measure it seems

In [2]:
# Importing libraries
import pandas as pd
import numpy as np

In [3]:
# reading Excel file
headcount = pd.read_excel('Demandv1.1.xlsx', 'Headcount',header=0)
#Headcount = Headcount[Headcount['Status'] == 'Bench']
#Headcount = Headcount.reset_index(drop = True)
demand_ly = pd.read_excel('Demandv1.1.xlsx', 'Demand Trend Last year',header=0)

In [4]:
# Available Information
import math

#Adding new attribute 'month of billable start'
headcount['month of billable'] = 'NA'
billable_mnth_index = headcount[headcount['Status'] == 'Billable'].index
headcount.loc[billable_mnth_index,'month of joining'] = 'December'
# Conver Employee Code to data type 'object'
headcount['Employee Code'] = headcount['Employee Code'].astype(object)
demand_ly['No. of New Hire Allocation'] = 0
demand_ly['No. of Bench Allocation'] = 0
demand_ly['Demand Met'] = 'No'
new_hire = 0
demand_ly['unmet_dmnd'] = 0

mnth_dict =  {'January':('January','February'), 'February': ('February','March'),
              'March':('March','April'), 'April':('April','May'), 'May': ('May','June'),
              'June': ('June','July') , 'July': ('July','August'), 'August': ('August','September'),
              'September': ('September','October'), 'October': ('October','November'), 
              'November': ('November','December'), 'December': ('December','December')}
rule = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8,
       'September':9, 'October':10, 'November':11, 'December':12}


unmet_dmnd = 0
bench_headcount = len(headcount[headcount['Status'] == 'Bench'])
incurred_expenses = bench_headcount*685 #for current year from jan
annual_avg_attrition = .20 # or 20 percent
incurred_opportunity_cost = 0 #for current year from jan
avg_headcount = (10000+12000)/2
annual_attrition_count = avg_headcount*(20/100)
fixed_monthly_attrition_count = math.ceil(annual_attrition_count/12)
opportunity_cost_peremployee_permonth = 900 # in USD
opportunity_cost = unmet_dmnd*opportunity_cost_peremployee_permonth
unmet_demand_cost_peremployee_permonth = 685 # in USD


In [5]:
# Capture Key Performance Constarints for Dashboard Creation and Display
# Create Datframe to store Captured Data
import pandas as pd
import numpy as np

def df_empty(columns, dtypes, index=None):
    assert len(columns)==len(dtypes)
    df = pd.DataFrame(index=index)
    for c,d in zip(columns, dtypes):
        df[c] = pd.Series(dtype=d)
    return df
col = ['Month','Attrition Count','YTD Expenses','Demand Count','Avl.Bench(Skill)','Allocated Bench',
               'Prop. New Hire','Prop. EOY Expense','Unmet Demand','Adj. New Hire',
               'Adj. EOY Expenses','YTD Opportunity Cost','Total Bench HeadCount',
              'Total Billable HeadCount','Total HeadCount']
datatyp=[np.string_,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64,np.int64]
df = df_empty(col,datatyp)

In [6]:
def DashBoard(List_Capture_Data_Performance,df) :
    data_entry = pd.DataFrame([List_Capture_Data_Performance], columns = col)
    df = df.append(data_entry, ignore_index=True)

    return df

In [7]:
#################
#
### Attrition ###
#
#################

# New Attribute : Calculate Tenure attribute and add to headcount info
# Assuming , Local date of joining is the start of tenure with the company

In [8]:
headcount['Local Date of Joining'] = pd.to_datetime(headcount['Local Date of Joining'])
headcount['year of joining'] = [date.year for date in headcount['Local Date of Joining']]
headcount['month of joining'] = [date.month for date in headcount['Local Date of Joining']]

# current year or prediction year
month = 1; year = 2019 # as headcount contains local joining year of 2018
# calculatin tenure as on 1st jan 2019
headcount['tenure in months'] = (2019 - headcount['year of joining']) * 12 + (headcount['month of joining'] - month)

In [9]:
# monthly attrition calculation

def attrition_execute(month,headcount,demand_ly,rule,fixed_monthly_attrition_count) :
    mnthly_attrition_pool = headcount[headcount['Status'] != 'Attrition']
    mnthly_attrition_pool = mnthly_attrition_pool[mnthly_attrition_pool['tenure in months']>12]
    attrition_index = np.random.choice(mnthly_attrition_pool.index,fixed_monthly_attrition_count)
    
    #Create Demand due to attrition
    #virtual_new_demand = Headcount.iloc[attrition_rows,:]
    for i in attrition_index :
        data_entry = pd.DataFrame([[month,1,headcount.loc[i,'SkillList'],headcount.loc[i,'Location'],'Exp Grade',
                               headcount.loc[i,'Market Unit'],
                                    'Skill Group','Attrition',0,0,'No',0]], columns = demand_ly.columns)
        demand_ly = demand_ly.append(data_entry, ignore_index=True)
        
    # convert status to attrition in headcount, instead of deleting preserving the data
    headcount.loc[attrition_index,'Status'] = 'Attrition'
    for i in headcount[headcount['Status'] != ' Attrition'].index :
        headcount.loc[i,'tenure in months'] = headcount.loc[i,'tenure in months'] + 1
    headcount = headcount.reset_index(drop = True)
    List_Capture_Data_Performance.append(headcount[headcount['Status'] == 'Attrition'].shape[0])
    
    return headcount, demand_ly

In [10]:
def Generate_Demand(df,month,demand_ly,rule,headcount,unmet_dmnd,incurred_expenses,opportunity_cost,
                    List_Capture_Data_Performance) :
    #Create monthy demand N+2 formula
    unmet_dmnd = 0 
        
    mnthly_demand = pd.DataFrame(demand_ly[demand_ly['Month DD Raised'].isin(mnth_dict[month])])
    mnthly_demand['Month DD'] = mnthly_demand['Month DD Raised'].replace(rule)
    mnthly_demand = mnthly_demand[mnthly_demand['Demand Met'] == 'No']
    mnthly_demand.sort_values(['Month DD','SkillList','Location'], inplace=True)
    
    incurred_expenses = bench_headcount*685 + incurred_expenses; List_Capture_Data_Performance.append(incurred_expenses)
    incurred_opportunity_cost = opportunity_cost
        
    for ind in (mnthly_demand.index) :
        skill = mnthly_demand.loc[ind,'SkillList']
        requirement = mnthly_demand.loc[ind,'No. of FTE Request Raised']
        location = mnthly_demand.loc[ind,'Location']
        practice = mnthly_demand.loc[ind,'Practice']
        month = month
        List_Capture_Data_Performance.append(requirement)
        
        headcount,demand_ly,new_hire,List_Capture_Data_Performance,opportunity_cost,incurred_expenses = Generate_Supply(skill,requirement, 
                                                       location, month,practice, headcount, 
                                                       demand_ly,ind,rule,unmet_dmnd,incurred_expenses,
                                                       incurred_opportunity_cost,List_Capture_Data_Performance,opportunity_cost)
        df = DashBoard(List_Capture_Data_Performance,df)
        m = List_Capture_Data_Performance[0]; a = List_Capture_Data_Performance[1]; e = List_Capture_Data_Performance[2]
        List_Capture_Data_Performance = []
        List_Capture_Data_Performance.append(m);List_Capture_Data_Performance.append(a);List_Capture_Data_Performance.append(e)
        
        
    return df,headcount,demand_ly,List_Capture_Data_Performance,incurred_expenses,opportunity_cost

In [11]:
from datetime import datetime
def Generate_Supply(skill,requirement, location, month,practice, headcount, demand_ly,ind,rule,unmet_dmnd,
                    incurred_expenses,incurred_opportunity_cost,List_Capture_Data_Performance,opportunity_cost) :
    mnthly_supply = headcount[headcount['Status'] == 'Bench']
    mnthly_supply = mnthly_supply[mnthly_supply['SkillList'] == skill]
    bench_total_cnt = mnthly_supply.shape[0]; List_Capture_Data_Performance.append(bench_total_cnt)
    
    if requirement > bench_total_cnt :
        bench = bench_total_cnt
    else :
        bench = requirement
   
    bench_index = mnthly_supply.index[0:bench]
    
    headcount.loc[bench_index,'Status'] = 'Billable' ; headcount.loc[bench_index,'month of billable'] = month
 
    demand_ly.loc[ind,'No. of Bench Allocation'] = bench; List_Capture_Data_Performance.append(bench)
    
    new_hire = requirement - bench
    List_Capture_Data_Performance.append(new_hire)
    
    new_hire,demand_ly,List_Capture_Data_Performance,opportunity_cost = Constrain_Exp_Head_Cal(opportunity_cost,new_hire,headcount,incurred_expenses,unmet_dmnd,incurred_opportunity_cost,ind,List_Capture_Data_Performance,demand_ly)
    
    
    if new_hire >0 :
        demand_ly.loc[ind,'No. of New Hire Allocation'] = new_hire
        new_hire_entry = pd.DataFrame([['Region','Employee Code','Last Name',datetime(2019,rule[month],1),
                             'Designation','Billable',practice,skill,location,month,rule[month],2019,
                                        rule[month]-1]], columns = headcount.columns)
        # New Hire Append to Headcount
        for j in range(new_hire) :
            headcount = headcount.append(new_hire_entry, ignore_index=True)
    else :
        demand_ly.loc[ind,'No. of New Hire Allocation'] = 0
    demand_ly.loc[ind,'Demand Met'] = 'Yes'
    
    List_Capture_Data_Performance.append(headcount[headcount['Status'] == 'Bench'].shape[0])
    List_Capture_Data_Performance.append(headcount[headcount['Status'] == 'Billable'].shape[0])
    List_Capture_Data_Performance.append(headcount[headcount['Status'] != 'Attrition'].shape[0])
    
    return headcount,demand_ly,new_hire,List_Capture_Data_Performance,opportunity_cost,incurred_expenses

In [12]:
def Constrain_Exp_Head_Cal(opportunity_cost,new_hire, headcount,incurred_expenses,unmet_dmnd,
                           incurred_opportunity_cost,ind,List_Capture_Data_Performance,demand_ly) :
    bench_headcount = len(headcount[headcount['Status'] == 'Bench'])
    billable_headcount = len(headcount[headcount['Status'] == 'Billable'])
    total_headcount = bench_headcount + billable_headcount+new_hire;
    remaining_mnth_cnt = 12-rule[month]
    expenses = bench_headcount*685*remaining_mnth_cnt +incurred_expenses ;List_Capture_Data_Performance.append(expenses)
    
    if expenses > 5760000 :
        unmet_dmnd = new_hire
        new_hire = 0
        demand_ly.loc[ind,'unmet_dmnd'] = unmet_dmnd
        # Creating Unmet Demand
        data_entry = pd.DataFrame([[demand_ly.loc[ind,'Month DD Raised'],
                                    unmet_dmnd,
                                    demand_ly.loc[ind,'SkillList'],
                                    demand_ly.loc[ind,'Location'],
                                    demand_ly.loc[ind,'Experience Grade'],
                                    demand_ly.loc[ind,'Practice'],
                                    demand_ly.loc[ind,'Skill Group'],
                                    demand_ly.loc[ind,'Demand Source'],
                                    0,0,'No',0]], columns = demand_ly.columns)
        demand_ly = demand_ly.append(data_entry, ignore_index=True)
    

    if total_headcount > 12000 :
        unmet_dmnd = new_hire
        new_hire = 0
        demand_ly.loc[ind,'unmet_dmnd'] = unmet_dmnd
        # Creating Unmet Demand
        data_entry = pd.DataFrame([[demand_ly.loc[ind,'Month DD Raised'],
                                unmet_dmnd,
                                demand_ly.loc[ind,'SkillList'],
                                demand_ly.loc[ind,'Location'],
                                demand_ly.loc[ind,'Experience Grade'],
                                demand_ly.loc[ind,'Practice'],
                                demand_ly.loc[ind,'Skill Group'],
                                demand_ly.loc[ind,'Demand Source'],
                                0,0,'No',0]], columns = demand_ly.columns)
        demand_ly = demand_ly.append(data_entry, ignore_index=True)
    
    List_Capture_Data_Performance.append(unmet_dmnd); List_Capture_Data_Performance.append(new_hire)
    
    expenses = bench_headcount*685*remaining_mnth_cnt +incurred_expenses;List_Capture_Data_Performance.append(expenses)
    opportunity_cost = unmet_dmnd*900 + opportunity_cost+incurred_opportunity_cost ;List_Capture_Data_Performance.append(opportunity_cost)
    

    return new_hire,demand_ly,List_Capture_Data_Performance,opportunity_cost

In [13]:
# Master Program
for month in rule:
    List_Capture_Data_Performance = []; List_Capture_Data_Performance.append(month)
    #incurred_expenses = exp_headcnt_cal(headcount,demand_ly,month,incurred_expenses)
    headcount, demand_ly = attrition_execute(month,headcount,demand_ly,rule,fixed_monthly_attrition_count)
    df,headcount,demand_ly,List_Capture_Data_Performance,incurred_expenses,opportunity_cost = Generate_Demand(df,month,demand_ly,rule,headcount,unmet_dmnd,incurred_expenses,opportunity_cost,List_Capture_Data_Performance)

In [14]:
df.to_csv('Dashboard.csv')